In [ ]:
#| default_exp components

%load_ext autoreload
%autoreload 2


# Transformers components to be assembled in a model

Work in progress

References:
- https://web.stanford.edu/class/cs224n/slides/cs224n-2022-lecture09-transformers.pdf

In [ ]:
# | export
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
from dataclasses import dataclass
from my_transformer.utils import save_model, load_model
from torch.utils.tensorboard import SummaryWriter
import math


device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# device = torch.device('cpu')
g = torch.Generator(device=device).manual_seed(42)

### Positional Encoding

As each word in a sentence simultaneously flows through the Transformer’s encoder/decoder stack, The model itself doesn’t have any sense of position/order for each word. Consequently, there’s still the need for a way to incorporate the order of the words into our model.
So we give the model some sense of position of the token in the sequence. 

Either we give the position as an input to the model or the model learns it.

#### Non learned positional embeddings

##### Potential solutions:

The first idea that might come to mind is to assign a number to each time-step within the [0, 1] range in which 0 means the first word and 1 is the last time-step. One of the problems it will introduce is that you can’t figure out how many words are present within a specific range. In other words, time-step delta doesn’t have consistent meaning across different sentences.

Another idea is to assign a number to each time-step linearly. That is, the first word is given “1”, the second word is given “2”, and so on. The problem with this approach is that not only the values could get quite large, but also our model can face sentences longer than the ones in training.

Ideally, the following criteria should be satisfied:

- It should output a unique encoding for each time-step (word’s position in a sentence)
- Distance between any two time-steps should be consistent across sentences with different lengths.
- Our model should generalize to longer sentences without any efforts. Its values should be bounded.
- It must be deterministic.

##### Proposed solutions:

The initial solution that was proposed isn’t a single number. Instead, it’s a d-dimensional vector that contains information about a specific position in a sentence. This vector, if not learned, is not integrated in the model.

$\begin{align}
  \vec{p_t}^{(i)} = f(t)^{(i)} & := 
  \begin{cases}
      \sin({\omega_k} . t),  & \text{if}\  i = 2k \\
      \cos({\omega_k} . t),  & \text{if}\  i = 2k + 1
  \end{cases}
\end{align} $

where $\omega_k = \frac{1}{n^{2k / d}}$

where : 
- L: sequence length
- t: position of token in input sequence
- d: dimension of positon embedding (same as token embedding)
- P(t,j): position function to map a position t in sequence to index (t,i) in positional matrix
- n: user defined scalar (ex: 10'000)
- i: index on the position embedding
- k: floor division (partie entière) of i by 2, so i = 2k (i is even - pair) or i=2k+1 (i is odd - impair), because we add a sin/cos pair on every two embedding slot

The frequency of sinusoidal is decreasing with the vector dimension (as i grows)

So we got a vector with pairs of sin/cosines for each frequency.

$\vec{p_t} = \begin{bmatrix} 
\sin({\omega_1}.t)\\ 
\cos({\omega_1}.t)\\ 
\\
\sin({\omega_2}.t)\\ 
\cos({\omega_2}.t)\\ 
\\
\vdots\\ 
\\
\sin({\omega_{d/2}}.t)\\ 
\cos({\omega_{d/2}}.t) 
\end{bmatrix}_{d \times 1}$

It's like the encoding of numbers in binary format: 

$
\begin{align}
  0: \ \ \ \ \color{orange}{\texttt{0}} \ \ \color{green}{\texttt{0}} \ \ \color{blue}{\texttt{0}} \ \ \color{red}{\texttt{0}} & & 
  8: \ \ \ \ \color{orange}{\texttt{1}} \ \ \color{green}{\texttt{0}} \ \ \color{blue}{\texttt{0}} \ \ \color{red}{\texttt{0}} \\
  1: \ \ \ \ \color{orange}{\texttt{0}} \ \ \color{green}{\texttt{0}} \ \ \color{blue}{\texttt{0}} \ \ \color{red}{\texttt{1}} & & 
  9: \ \ \ \ \color{orange}{\texttt{1}} \ \ \color{green}{\texttt{0}} \ \ \color{blue}{\texttt{0}} \ \ \color{red}{\texttt{1}} \\ 
  2: \ \ \ \ \color{orange}{\texttt{0}} \ \ \color{green}{\texttt{0}} \ \ \color{blue}{\texttt{1}} \ \ \color{red}{\texttt{0}} & & 
  10: \ \ \ \ \color{orange}{\texttt{1}} \ \ \color{green}{\texttt{0}} \ \ \color{blue}{\texttt{1}} \ \ \color{red}{\texttt{0}} \\ 
  3: \ \ \ \ \color{orange}{\texttt{0}} \ \ \color{green}{\texttt{0}} \ \ \color{blue}{\texttt{1}} \ \ \color{red}{\texttt{1}} & & 
  11: \ \ \ \ \color{orange}{\texttt{1}} \ \ \color{green}{\texttt{0}} \ \ \color{blue}{\texttt{1}} \ \ \color{red}{\texttt{1}} \\ 
  4: \ \ \ \ \color{orange}{\texttt{0}} \ \ \color{green}{\texttt{1}} \ \ \color{blue}{\texttt{0}} \ \ \color{red}{\texttt{0}} & & 
  12: \ \ \ \ \color{orange}{\texttt{1}} \ \ \color{green}{\texttt{1}} \ \ \color{blue}{\texttt{0}} \ \ \color{red}{\texttt{0}} \\
  5: \ \ \ \ \color{orange}{\texttt{0}} \ \ \color{green}{\texttt{1}} \ \ \color{blue}{\texttt{0}} \ \ \color{red}{\texttt{1}} & & 
  13: \ \ \ \ \color{orange}{\texttt{1}} \ \ \color{green}{\texttt{1}} \ \ \color{blue}{\texttt{0}} \ \ \color{red}{\texttt{1}} \\
  6: \ \ \ \ \color{orange}{\texttt{0}} \ \ \color{green}{\texttt{1}} \ \ \color{blue}{\texttt{1}} \ \ \color{red}{\texttt{0}} & & 
  14: \ \ \ \ \color{orange}{\texttt{1}} \ \ \color{green}{\texttt{1}} \ \ \color{blue}{\texttt{1}} \ \ \color{red}{\texttt{0}} \\
  7: \ \ \ \ \color{orange}{\texttt{0}} \ \ \color{green}{\texttt{1}} \ \ \color{blue}{\texttt{1}} \ \ \color{red}{\texttt{1}} & & 
  15: \ \ \ \ \color{orange}{\texttt{1}} \ \ \color{green}{\texttt{1}} \ \ \color{blue}{\texttt{1}} \ \ \color{red}{\texttt{1}} \\
\end{align}$

where last bit is alternating on every number and previous on every 2 numbers, and so on. So instead of using bits (which would be a waste of space), we use continuous float variant with sinuosidal functions..

## References

- https://erdem.pl/2021/05/understanding-positional-encoding-in-transformers
- https://kazemnejad.com/blog/transformer_architecture_positional_encoding/
- https://kikaben.com/transformers-positional-encoding/
- http://nlp.seas.harvard.edu/annotated-transformer/#positional-encoding
- https://towardsdatascience.com/master-positional-encoding-part-i-63c05d90a0c3



## positional encoder must be dynamic for any lenght (up to max length) provided while generating tokens.

In [ ]:
# | export


class PositionalEncoder(nn.Module):
    """Module to encode position in a transformer like model.


    Args:
        max_seq_len (int): max length of sequence, aka L
        embedding_dim (int): dimension of embeddings in model, aka d
        dropout (float):  dropout rate. 0. for no dropout.
        is_learned (bool): true if the position is learned through gradient descent or given (non differentiable) as defined in original paper Attention is all you need, https://arxiv.org/abs/1706.03762
        n (int): user defined scalar set by default to 10000 as in paper
    """

    def positional_encoding(self):
        pos = torch.arange(0, self.max_seq_len).repeat(self.embedding_dim, 1)
        i = torch.arange(0, self.embedding_dim)
        k = i // 2
        wt = pos.T / (self.n ** (2 * k / self.embedding_dim))
        sin = torch.sin(wt)
        cos = torch.cos(wt)
        pe = torch.zeros((self.max_seq_len, self.embedding_dim))
        pe[:, 0::2] = sin[:, 0::2]
        pe[:, 1::2] = cos[:, 1::2]
        return pe

    def __init__(
        self, max_seq_len: int, embedding_dim: int, dropout: float = 0.0, is_learned: bool = True, n: int = 10000
    ) -> None:
        super().__init__()
        self.max_seq_len = max_seq_len
        self.embedding_dim = embedding_dim
        self.is_learned = is_learned
        self.n = n
        self.dropout = nn.Dropout(p=dropout)
        if self.is_learned:
            pos = torch.arange(0, self.max_seq_len, dtype=torch.long).unsqueeze(0)  # shape (1, max_seq_len)
            self.register_buffer("pos", pos)
            self.pos_embedding = nn.Embedding(max_seq_len, embedding_dim)
        else:
            pos_encodings = self.positional_encoding().unsqueeze(0)
            pos_encodings.requires_grad_(False)
            # a buffer is a state in module which is not a parameter (learned)
            self.register_buffer("pos_encodings", pos_encodings)

    def forward(self, x):
        B, T, C = x.shape
        # we only need the first T positions
        if self.is_learned:
            x = x + self.pos_embedding(self.pos)[:, :T]
        else:
            x = x + self.pos_encodings[:, :T]
        return self.dropout(x)

In [ ]:
# non learned positional encoding
seq_len = 20
embedding_dim = 50
pos_enc = PositionalEncoder(max_seq_len=seq_len, embedding_dim=embedding_dim, dropout=0.1, is_learned=False)
pos_enc.to(device)
pos_enc.positional_encoding()
x = torch.zeros(1, seq_len, embedding_dim, device=device)
pe = pos_enc.forward(x)
np_pe = pe.cpu().detach().numpy()
plt.pcolormesh(np_pe[0].T, cmap="RdBu")
plt.xlabel("Position")
plt.ylabel("Depth")
plt.colorbar()
plt.show()

## Attention mechanisms

### attention function

In [ ]:
# | export
def unidirectional_mask(seq_len: int):
    inverse_mask = torch.triu(torch.ones((1, seq_len, seq_len)), diagonal=1).type(torch.uint8)
    mask = inverse_mask == 0
    return mask

In [ ]:
mask = unidirectional_mask(20)
plt.imshow(mask[0])
plt.colorbar()
plt.show()

## Attention mechanism
Based on formal algorithm for transformers (https://arxiv.org/abs/2207.09238) where it can be used for different attention architectures like 
- encoder-decoder/seq-to-seq (very first transformer), 
- encoder-only (bert), 
- decoder-only (gpt-*, gopher). 

### Encoder-decoder architecture use 2 sequences as input
- context sequence of tokens (z), which is used to encode context as a vector per token with bidirectional attention
- primary sequence of tokens (x), which is used to decode both the encoded context and a masked primary sequence (tokens in primary sequence that precedes current). This is used to train a translator from FR to EN for instance. FR tokens are the context sequence and EN tokens are the primary sequence (target)
 
### Encoder only architecture use 1 primary sequence as input
Given a primary input sequence (x) with some tokens masked out, the goal is to recover the masked tokens. the goal is to learn a generally usefull representation of text. Uses a bidirectional attention

### Decoder only architecture use 1 primary sequence as input
Autoregressive language modelling where the goal is to predict the next token of a primary token sequence (x). Uses a unidirectional attention. 

In [ ]:
# | export
def attention(
    query: torch.Tensor,
    key: torch.Tensor,
    value: torch.Tensor,
    mask: torch.Tensor = None,
    dropout: torch.nn.Module = None,
    verbose: bool = False,
) -> tuple[torch.Tensor, torch.Tensor]:
    # is the size of atttention (=d//nb heads, where d is the size of attn model)
    d_k = query.size(-1)
    # scores = (query @ key.transpose(-2,-1)) /(torch.sqrt(torch.tensor(d_k, requires_grad=False)))
    scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        # mask scores with -inf when mask is False. If we masked_fill(mask, -inf), it would mask when mask ==1 (is true). We want to mask when it's 0 (false).
        scores = scores.masked_fill(mask == 0, float("-inf"))
    if verbose:
        print("scores, mask", mask is not None, scores)
    soft = F.softmax(scores, dim=-1)
    if verbose:
        print("soft, mask", mask is not None, soft)
    if dropout is not None:
        soft = dropout(soft)
    atn = soft @ value
    if verbose:
        print("atn, mask", mask is not None, atn)

    assert (
        atn.shape == query.shape
    ), f"atn shape {atn.shape} should be the same as input tensors key, query and value shapes. Query shape: {query.shape}"
    return atn, soft

In [ ]:
h = 5  # nb of heads
d_k = embedding_dim // h
batch_size = 2
# shape is batch, h, seq_len, d_k
Q = torch.randn((batch_size, h, seq_len, d_k), generator=g, device=device, requires_grad=True)
K = torch.randn((batch_size, h, seq_len, d_k), generator=g, device=device, requires_grad=True)
V = torch.randn((batch_size, h, seq_len, d_k), generator=g, device=device, requires_grad=True)
mask = unidirectional_mask(seq_len).to(device)
attn, attn_weight = attention(Q, K, V, mask=mask, dropout=torch.nn.Dropout(0.0))
attn_no_mask, attn_weight_no_mask = attention(Q, K, V, dropout=torch.nn.Dropout(0.0))

plt.imshow(attn[0][0].cpu().detach().numpy())
plt.colorbar()
plt.show()

np_attn_weight = attn_weight.cpu()
np_sum = torch.sum(np_attn_weight, dim=1).cpu().detach().numpy()
plt.imshow(np_sum[0])
plt.colorbar()
plt.show()


target = torch.rand(*attn.shape).to(device)
print(target.shape, pe.shape)
loss = torch.norm(attn - target)
loss.retain_grad()
attn.retain_grad()
loss.backward()
loss.item(), attn.grad.shape, loss.grad

In [ ]:
# | export


class MultiHeadAttention(nn.Module):
    """Multihead attention module as defined in Formal algorithm for transformers (https://arxiv.org/abs/2207.09238)
    It can be used for different attention architectures like encoder-decoder/seq-to-seq (very first transformer),
    encoder-only (bert), decoder-only (gpt-*, gopher).

    It splits weights into h heads.
    """

    def __init__(self, d: int, h: int, dropout: float = 0.0, bias: bool = True):
        super().__init__()
        # g = torch.Generator().manual_seed(42)
        self.d = d  # dim of attention
        self.h = h
        assert (
            d % h == 0
        ), f"dim of attention {d} must be a multiple of nb of heads {h}. d_k {d//h} is the model dim per head, because we split by nb of heads."
        self.d_k = d // h  # dim of attention on one head.
        self.wq = nn.Linear(d, d, bias=bias)
        self.wk = nn.Linear(d, d, bias=bias)
        self.wv = nn.Linear(d, d, bias=bias)
        self.wo = nn.Linear(d, d, bias=bias)  # linear transform by output weight matrix.
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor, z: torch.Tensor, mask: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
        """_summary_

        Args:
            x (torch.Tensor): primary sequence
            z (torch.Tensor): context sequence, only for encoder-decoder architecture.
            mask (torch.Tensor, optional): _description_. Defaults to None.
        """
        b, l, e = x.size()
        if z is None:
            z = x
        q = self.wq(x)
        k = self.wk(z)
        v = self.wk(z)
        # shape has to be batch, h, seq, d_k (d//h)
        # first view to : b,l,h,d_k , then transpose h and l so we got per head q,k,v
        q = q.view(b, l, self.h, self.d_k).transpose(1, 2)
        k = k.view(b, l, self.h, self.d_k).transpose(1, 2)
        v = v.view(b, l, self.h, self.d_k).transpose(1, 2)

        # attention has to be done on each head.
        mh_attn, mh_attn_weight = attention(q, k, v, mask=mask, dropout=self.dropout)
        # print(mh_attn.shape, mh_attn_weight.shape)
        # we need to create a contiguous memory space for tensor after transpose so we can apply view.
        concat_attn = mh_attn.transpose(2, 1).contiguous().view(b, l, self.h * self.d_k)
        concat_attn_weight = torch.sum(mh_attn_weight, dim=1)
        assert concat_attn.size() == (b, l, e)
        # apply linear layer on concatenated attention.
        output = self.dropout(self.wo(concat_attn))
        return output, concat_attn_weight

In [ ]:
# check it can compute gradientbatch_size=2
primary_seq = torch.rand((batch_size, seq_len, embedding_dim)).to(device)
ctx_seq = torch.rand((batch_size, seq_len, embedding_dim)).to(device)
mh = MultiHeadAttention(d=embedding_dim, h=5, dropout=0.0, bias=False).to(device)
mh_attn, mh_attn_weight = mh(primary_seq, ctx_seq, mask=mask)

# mh_attn.retain_grad()
target = torch.rand(*mh_attn.shape).to(device)
print(target.shape, pe.shape)
loss = torch.norm(mh_attn - target)
mh_attn.retain_grad()
loss.retain_grad()
loss.backward()
loss.item(), mh_attn.grad.shape, loss.grad

In [ ]:
batch_size = 2
primary_seq = torch.rand((batch_size, seq_len, embedding_dim)).to(device)
ctx_seq = torch.rand((batch_size, seq_len, embedding_dim)).to(device)
mh = MultiHeadAttention(d=embedding_dim, h=5, dropout=0.0, bias=False).to(device)
mh_attn, mh_attn_weight = mh(primary_seq, ctx_seq, mask=mask)

plt.imshow(mh_attn[0].cpu().detach().numpy())
plt.colorbar()
plt.show()

plt.imshow(mh_attn_weight[0].cpu().detach().numpy())
plt.colorbar()
plt.show()

In [ ]:
# test with multihead_attn = nn.MultiheadAttention(embed_dim, num_heads) so that result is the same.
# the graph output should look the same, even if they are not exactly the same due to different initialization
nn_mh = nn.MultiheadAttention(embed_dim=embedding_dim, num_heads=5, dropout=0.0, batch_first=True).to(device)
nn_mh_atn, nn_mh_atn_weights = nn_mh(primary_seq, ctx_seq, ctx_seq, attn_mask=(mask[0] == 0))
plt.imshow(nn_mh_atn[0].cpu().detach().numpy())
plt.colorbar()
plt.show()

plt.imshow(nn_mh_atn_weights[0].cpu().detach().numpy())
plt.colorbar()
plt.show()

# check it can compute gradient
nn_mh_atn.retain_grad()
target = torch.rand(*nn_mh_atn.shape).to(device)
loss = torch.norm(nn_mh_atn - target)
loss.retain_grad()
loss.backward()
loss.item()
loss.grad, nn_mh_atn.grad.shape

## Layer normalization


In [ ]:
# | export
class LayerNormalization(nn.Module):
    def __init__(self, d: int, eps: float = 1e-05, use_torch_implem: bool = True) -> None:
        super().__init__()
        self.d = d
        self.eps = eps
        self.use_torch_implem = use_torch_implem
        self.gamma = torch.nn.Parameter(torch.ones(d, dtype=torch.float))  # scale
        self.beta = torch.nn.Parameter(torch.zeros(d, dtype=torch.float))  # offset

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if self.use_torch_implem:
            # we normalize across features for each example and seq item.
            normalized_shape = x.size()[-1:]
            b, l, e = x.size()
            x_hat = F.layer_norm(x, normalized_shape=[e], weight=self.gamma, bias=self.beta)
        else:
            # normalization across features (independently) for each sample. We compute mean and var on the last 2 axis, so we have it per sampel
            mean = x.mean((-1), keepdim=True)  # .unsqueeze(-1)
            var = x.var((-1), keepdim=True)  # .unsqueeze(-1)
            x_hat = torch.mul((x - mean) / (torch.sqrt(var) + self.eps), self.gamma) + self.beta

        return x_hat

In [ ]:
# check implement is close to pytorch one.
x = torch.rand((batch_size, seq_len, embedding_dim)).to(device)
layer_norm = LayerNormalization(d=embedding_dim, use_torch_implem=False).to(device)
x_hat = layer_norm(x)

torch_layer_norm = nn.LayerNorm(embedding_dim, eps=1e-05, elementwise_affine=True).to(device)
torch_x_hat = torch_layer_norm(x)
assert torch.allclose(x_hat, torch_x_hat, rtol=0.02, atol=0.0), "my implementation and torch should be close"

target = torch.rand(*x_hat.shape).to(device)
print(target.shape, x_hat.shape)
loss = torch.norm(x_hat - target)
loss.retain_grad()
x_hat.retain_grad()
loss.backward()
loss.item(), x_hat.grad.shape, loss.grad

## Decoder transformer

Architecture used by GPT-*, Gopher, where it want to predict the next token given previous ones. 

In [ ]:
# | export


class DecoderLayer(nn.Module):
    def __init__(
        self, model_size: int, nb_heads: int = 1, dropout: float = 0.0, bias: bool = True, mlp_factor=4
    ) -> None:
        super().__init__()
        self.layer_norm1 = LayerNormalization(model_size)
        self.attn = MultiHeadAttention(d=model_size, h=nb_heads, dropout=dropout, bias=bias)
        self.mlp1 = nn.Linear(model_size, mlp_factor * model_size, bias=True)
        self.mlp2 = nn.Linear(mlp_factor * model_size, model_size, bias=True)
        self.activation = torch.nn.GELU()
        self.layer_norm2 = LayerNormalization(model_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        b, l, d = x.size()
        norm_x = self.layer_norm1(x)
        mask = unidirectional_mask(seq_len=l)
        attn_x = x + self.attn(norm_x, z=None, mask=None)[0]

        norm_attn_x = self.layer_norm2(attn_x)
        lin1 = self.activation(self.mlp1(norm_attn_x))
        x = x + self.dropout(self.mlp2(lin1))
        return x

In [ ]:
x = torch.rand((batch_size, seq_len, embedding_dim)).to(device)
dl = DecoderLayer(embedding_dim, nb_heads=1, dropout=0.1).to(device)
dlo = dl(x)

plt.imshow(x[0].cpu().detach().numpy())
plt.colorbar()
plt.show()

plt.imshow(dlo[0].cpu().detach().numpy())
plt.colorbar()
plt.show()

In [ ]:
# | export


class DecoderTransformer(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        max_seq_len: int,
        model_size: int,
        nb_heads: int = 1,
        nb_layers: int = 1,
        dropout: float = 0.0,
        bias: bool = True,
    ) -> None:
        super().__init__()
        self.tok_emb = nn.Embedding(vocab_size, embedding_dim=model_size)
        self.pos_enc = PositionalEncoder(
            max_seq_len=max_seq_len, embedding_dim=model_size, dropout=dropout, is_learned=True
        )
        self.layers = nn.ModuleList(
            [
                DecoderLayer(model_size=model_size, nb_heads=nb_heads, dropout=dropout, bias=bias)
                for i in range(nb_layers)
            ]
        )
        self.layer_norm = LayerNormalization(model_size)
        self.unembedding = nn.Linear(model_size, vocab_size)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        seq_len = x.size()[1]
        emb = self.tok_emb(x)
        x = self.pos_enc(emb)
        for layer in self.layers:
            x = layer(x)
        x = self.layer_norm(x)
        logits = self.unembedding(x)
        return logits

In [ ]:
# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device = "cpu"
dec = DecoderTransformer(vocab_size=6, max_seq_len=5, model_size=10, nb_heads=1).to(device)
x = torch.tensor([[0, 1, 2, 3, 4], [1, 2, 3, 4, 5]], dtype=torch.int, device=device)
y_hat = dec(x)
y_hat.shape

target = torch.rand(*y_hat.shape).to(device)
print(target.shape, y_hat.shape)
loss = nn.CrossEntropyLoss()
loss = loss(y_hat, target)
loss.retain_grad()
y_hat.retain_grad()
loss

## Training and dataset

In [ ]:
# with open("../dataset/shakespeare.txt") as f:
#    text = f.read()

df = pd.read_csv("../dataset/bob_dylan_lyrics.csv")
text = "\n\n".join(df.lyrics.values.tolist())

In [ ]:
class CharDataset(Dataset):
    def __init__(self, text: str, mode: str = "train", seq_len: int = 100, device: str = "cpu"):
        assert mode in ["train", "test"], "Mode must be 'train' or 'test'"
        self.mode = mode
        self.device = device
        self.test_start_pos = int(0.8 * len(text))
        self.train_text = text[: self.test_start_pos]
        self.test_text = text[self.test_start_pos :]

        self.seq_len = seq_len
        self.vocab = list(set(text))
        self.vocab_size = len(self.vocab)
        self.char_to_idx = {c: i for i, c in enumerate(self.vocab)}
        self.idx_to_char = {i: c for i, c in enumerate(self.vocab)}

        self.train_encoded = torch.tensor(self.encode(self.train_text), dtype=torch.long, device=device)
        self.test_encoded = torch.tensor(self.encode(self.test_text), dtype=torch.long, device=device)

    def __len__(self):
        if self.mode == "train":
            data = self.train_encoded
        else:
            data = self.test_encoded
        return len(data) // self.seq_len

    def __getitem__(self, idx):
        if self.mode == "train":
            data = self.train_encoded
        else:
            data = self.test_encoded

        data_len = data.shape[0]
        i = idx * self.seq_len
        if i >= data_len:
            raise ValueError(f"idx {idx} bigger than {self.mode} data length {data_len}")
        x = data[i : i + self.seq_len]
        y = data[i + 1 : i + self.seq_len + 1]
        x = x.to(device)
        y = y.to(device)
        return x, y

    def decode(self, x):
        return "".join([self.idx_to_char[i] for i in x])

    def encode(self, x):
        return [self.char_to_idx[c] for c in x]

    def get_vocab(self):
        return self.vocab

    def get_vocab_size(self):
        return self.vocab_size

In [ ]:
cds = CharDataset(text, seq_len=10)
l = len(cds)
x, y = cds[l]
assert isinstance(x, torch.Tensor)
assert isinstance(y, torch.Tensor)
try:
    cds[l + 1]
    assert False, "it should"
except ValueError as e:
    assert True, "it should raise a value error if bigger than len of dataset"

batch_size = 8
seq_len = 10
train_ds = CharDataset(text, seq_len=seq_len, device=device)
test_ds = CharDataset(text, seq_len=seq_len, mode="test", device=device)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=True)
test_x, test_y = next(iter(test_dl))
assert test_x.shape == (batch_size, seq_len)
assert test_y.shape == (batch_size, seq_len)

In [ ]:
class DecoderModel(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        seq_len: int,
        model_size: int,
        nb_heads: int = 1,
        nb_layers: int = 1,
        dropout: float = 0.1,
    ) -> None:
        super().__init__()
        self.seq_len = seq_len
        self.model_size = model_size
        self.vocab_size = vocab_size
        self.token_emb = torch.nn.Embedding(vocab_size, model_size)
        self.pos_enc = PositionalEncoder(
            max_seq_len=seq_len, embedding_dim=model_size, dropout=dropout, is_learned=True
        )
        self.layers = nn.Sequential(
            *[
                DecoderLayer(model_size=model_size, nb_heads=nb_heads, dropout=dropout, bias=True)
                for i in range(nb_layers)
            ]
        )
        self.layer_norm = LayerNormalization(model_size)
        self.lm_head = torch.nn.Linear(model_size, vocab_size)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_emb(idx)
        x = self.pos_enc(tok_emb)
        assert x.shape == (B, T, self.model_size)
        # it changes on each iteration.
        # pos_emb = self.pos_emb(torch.arange(T, device=device))
        # x = tok_emb + pos_emb  # B, T, C
        # print(f"x shape: {x.shape}")
        x = self.layers(x)
        x = self.layer_norm(x)
        logits = self.lm_head(x)
        # print(f"logits shape: {logits.shape}")
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # we take the last seq_len tokens as condition
            idx_cond = idx[:, -self.seq_len :]
            logits, _ = self(idx_cond)

            logits = logits[:, -1, :]

            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)

        return idx


torch.manual_seed(1337)
device = "mps"
batch_size = 16
seq_len = 10
model_size = 128

train_ds = CharDataset(text, seq_len=seq_len, device=device)
test_ds = CharDataset(text, seq_len=seq_len, mode="test", device=device)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=True)
dls = {"train": train_dl, "test": test_dl}

vocab_size = train_ds.get_vocab_size()

model = DecoderModel(vocab_size=vocab_size, seq_len=seq_len, model_size=model_size, nb_heads=4, nb_layers=1).to(device)
xb, yb = next(iter(test_dl))
train_logit, train_loss = model(xb, yb)
train_loss


In [ ]:
# | export
# trainer class for pytorch model that encapsulate training loop


class Trainer:
    def __init__(
        self,
        model: nn.Module,
        optimizer: torch.optim.Optimizer,
        train_dl: DataLoader,
        test_dl: DataLoader,
        eval_interval: int = 500,
        eval_iters: int = 200,
        max_iters: int = 5000,
        do_save_model: bool = True,
        model_version: str = "",
        device: str = "cpu",
    ) -> None:
        self.model = model
        self.optimizer = optimizer
        self.train_dl = train_dl
        self.test_dl = test_dl
        self.eval_interval = eval_interval
        self.eval_iters = eval_iters
        self.max_iters = max_iters
        self.do_save_model = do_save_model
        self.model_version = model_version
        self.device = device
        self.writer = SummaryWriter(f"../runs/my_gpt_{self.model_version}")

    @torch.no_grad()
    def estimate_loss(self, i: int):
        out = {}
        self.model.eval()
        for split in ["train", "test"]:
            dl = self.train_dl if split == "train" else self.test_dl
            losses = torch.zeros(self.eval_iters)
            for k in range(self.eval_iters):
                X, Y = next(iter(dl))
                logits, loss = self.model(X, Y)
                losses[k] = loss.item()
            loss_mean = losses.mean()
            out[split] = loss_mean
            self.writer.add_scalar(f"{split} loss", loss_mean, i)
        self.model.train()
        return out

    def train(self):
        for i in range(self.max_iters):
            # every once in a while evaluate the loss on train and val sets
            if i % self.eval_interval == 0:
                losses = self.estimate_loss(i)
                print(f"step {i}: train loss {losses['train']:.4f}, val loss {losses['test']:.4f}")

            # sample a batch of data
            xb, yb = next(iter(self.train_dl))

            logits, loss = self.model(xb, yb)
            self.optimizer.zero_grad(set_to_none=True)
            loss.backward()
            self.optimizer.step()

        if self.do_save_model:
            save_model(self.model, self.model_version)


In [ ]:
# create a PyTorch optimizer
model_version = "0.3"
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-2)
tr = Trainer(
    model,
    optimizer,
    train_dl,
    test_dl,
    eval_interval=500,
    eval_iters=200,
    max_iters=5001,
    do_save_model=True,
    model_version=model_version,
    device=device,
)
tr.train()

In [ ]:
do_load: bool = False
if do_load:
    model = load_model(model_version)

print("-----")
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(train_ds.decode(model.generate(context, max_new_tokens=200)[0].tolist()))


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export("./components.ipynb")

In [ ]:
max_iters = 5000
eval_interval = 500
learning_rate = 1e-2
eval_iters = 200
do_save_model: bool = True


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "test"]:
        dl = dls[split]
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = next(iter(dl))
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for i in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if i % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {i}: train loss {losses['train']:.4f}, val loss {losses['test']:.4f}")

    # sample a batch of data
    xb, yb = next(iter(train_dl))

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

if do_save_model:
    model_version = "0.2"
    save_model(model, model_version)


context = torch.zeros((1, 1), dtype=torch.long, device=device)
print("-----")
print(train_ds.decode(model.generate(context, max_new_tokens=100)[0].tolist()))


## Appendices

In [ ]:
import pickle


class AKSortDataset(Dataset):
    """
    Dataset for the Sort problem. E.g. for problem length 6:
    Input: 0 0 2 1 0 1 -> Output: 0 0 0 1 1 2
    Which will feed into the transformer concatenated as:
    input:  0 0 2 1 0 1 0 0 0 1 1
    output: I I I I I 0 0 0 1 1 2
    where I is "ignore", as the transformer is reading the input sequence
    """

    def __init__(self, split, length=6, num_digits=3):
        assert split in {"train", "test"}
        self.split = split
        self.length = length
        self.num_digits = num_digits

    def __len__(self):
        return 10000  # ...

    def get_vocab_size(self):
        return self.num_digits

    def get_block_size(self):
        # the length of the sequence that will feed into transformer,
        # containing concatenated input and the output, but -1 because
        # the transformer starts making predictions at the last input element
        return self.length * 2 - 1

    def __getitem__(self, idx):
        # use rejection sampling to generate an input example from the desired split
        while True:
            # generate some random integers
            inp = torch.randint(self.num_digits, size=(self.length,), dtype=torch.long)
            # half of the time let's try to boost the number of examples that
            # have a large number of repeats, as this is what the model seems to struggle
            # with later in training, and they are kind of rate
            if torch.rand(1).item() < 0.5:
                if inp.unique().nelement() > self.length // 2:
                    # too many unqiue digits, re-sample
                    continue
            # figure out if this generated example is train or test based on its hash
            h = hash(pickle.dumps(inp.tolist()))
            inp_split = "test" if h % 4 == 0 else "train"  # designate 25% of examples as test
            if inp_split == self.split:
                break  # ok

        # solve the task: i.e. sort
        sol = torch.sort(inp)[0]

        # concatenate the problem specification and the solution
        cat = torch.cat((inp, sol), dim=0)

        # the inputs to the transformer will be the offset sequence
        x = cat[:-1].clone()
        y = cat[1:].clone()
        # we only want to predict at output locations, mask out the loss at the input locations
        y[: self.length - 1] = -1
        return x, y


# print an example instance of the dataset
train_dataset = AKSortDataset("train")

x, y = train_dataset[0]
# for a, b in zip(x,y):
#    print(int(a),int(b))
x, y

In [ ]:
# min gpt implem
class CausalSelfAttention(nn.Module):
    """
    A vanilla multi-head masked self-attention layer with a projection at the end.
    It is possible to use torch.nn.MultiheadAttention here but I am including an
    explicit implementation here to show that there is nothing too scary here.
    """

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        # regularization
        self.attn_dropout = nn.Dropout(config.attn_pdrop)
        self.resid_dropout = nn.Dropout(config.resid_pdrop)
        # causal mask to ensure that attention is only applied to the left in the input sequence
        self.register_buffer(
            "bias",
            torch.tril(torch.ones(config.block_size, config.block_size)).view(
                1, 1, config.block_size, config.block_size
            ),
        )
        self.n_head = config.n_head
        self.n_embd = config.n_embd

    def forward(self, x):
        B, T, C = x.size()  # batch size, sequence length, embedding dimensionality (n_embd)
        print(B, T, C)
        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        print(self.c_attn(x).shape)
        q, k, v = self.c_attn(x).split(self.n_embd, dim=2)  # takes group of n_embd along dim 2.
        print(q.shape, k.shape, v.shape)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)  # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)  # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)  # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        # what is k.size(-1) ? d_k ?
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float("-inf"))
        att = F.softmax(att, dim=-1)
        att = self.attn_dropout(att)
        y = att @ v  # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C)  # re-assemble all head outputs side by side

        # output projection
        y = self.resid_dropout(self.c_proj(y))
        return y


@dataclass
class Cfg:
    n_embd = 50
    n_head = 5
    attn_pdrop = 0.0
    resid_pdrop = 0.0
    block_size = 1


x = torch.rand((1, seq_len, embedding_dim)).to(device)
cs = CausalSelfAttention(Cfg()).to(device)
cs_attn = cs(x)
plt.imshow(cs_attn[0].cpu().detach().numpy())
plt.colorbar()
plt.show()